In [ ]:
!pip install ultralytics kaggle opencv-python-headless matplotlib pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.0/871.0 kB 11.9 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# Upload kaggle.json
files.upload()

# Move kaggle.json to the correct directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the car plate detection dataset
!kaggle datasets download -d andrewmvd/car-plate-detection

# Unzip the dataset into car_plate_data directory
!unzip -o car-plate-detection.zip -d car_plate_data

# List files inside the unzipped directory
!ls car_plate_data | head -10


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Saving kaggle-1.json to kaggle-1.json
mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection
License(s): CC0-1.0
 99% 201M/203M [00:10<00:00, 23.7MB/s]
100% 203M/203M [00:11<00:00, 19.3MB/s]
Archive:  car-plate-detection.zip
  inflating: car_plate_data/annotations/Cars0.xml  
  inflating: car_plate_data/annotations/Cars1.xml  
  inflating: car_plate_data/annotations/Cars10.xml  
  inflating: car_plate_data/annotations/Cars100.xml  
  inflating: car_plate_data/annotations/Cars101.xml  
  inflating: car_plate_data/annotations/Cars102.xml  
  inflating: car_plate_data/annotations/Cars103.xml  
  inflating: car_plate_data/annotations/Cars104.xml  
  inflating: car_plate_data/annotations/Cars105.xml  
  inflating: car_plate_data/annotations/Cars106.xml  
  inflating: car_plate_data/annotations/Cars107.xml  
  inflating: car_plate_data/annot

In [ ]:
# Create the correct train directories for images and labels
!mkdir -p car_plate_data/images/train
!mkdir -p car_plate_data/labels/train

# Move all images to the train folder
!mv car_plate_data/images/*.png car_plate_data/images/train/

# Move all annotations (labels) to the train folder
!mv car_plate_data/annotations/*.xml car_plate_data/labels/train/

# Check the actual path for the images
!ls /content/car_plate_data/images/train | head -5

# Check the actual path for the labels
!ls /content/car_plate_data/labels/train | head -5


Cars0.png
Cars100.png
Cars101.png
Cars102.png
Cars103.png
Cars0.xml
Cars100.xml
Cars101.xml
Cars102.xml
Cars103.xml


In [ ]:
import os
import xml.etree.ElementTree as ET

# Paths to your annotations and labels
annotations_path = "car_plate_data/labels/train"
images_path = "car_plate_data/images/train"

# Create a function to convert Pascal VOC XML to YOLO txt format
def convert_voc_to_yolo(xml_file, output_txt_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image width and height
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    with open(output_txt_file, 'w') as f:
        for obj in root.iter('object'):
            class_id = 0  # Since we only have one class (license plate)
            xmlbox = obj.find('bndbox')
            xmin = int(xmlbox.find('xmin').text)
            xmax = int(xmlbox.find('xmax').text)
            ymin = int(xmlbox.find('ymin').text)
            ymax = int(xmlbox.find('ymax').text)

            # Convert to YOLO format
            x_center = (xmin + xmax) / (2.0 * width)
            y_center = (ymin + ymax) / (2.0 * height)
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            # Write the YOLO annotation
            f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

# Convert all XML files to YOLO txt format
for xml_file in os.listdir(annotations_path):
    if xml_file.endswith('.xml'):
        txt_file = os.path.join(annotations_path, xml_file.replace('.xml', '.txt'))
        convert_voc_to_yolo(os.path.join(annotations_path, xml_file), txt_file)

print("Conversion completed.")

# List the first 5 files in the labels folder
!ls car_plate_data/labels/train | head -5



Conversion completed.
Cars0.txt
Cars0.xml
Cars100.txt
Cars100.xml
Cars101.txt


In [ ]:
dataset_yaml = """
train: /content/car_plate_data/images/train
val: /content/car_plate_data/images/train  # Use train folder as val temporarily

nc: 1  # Number of classes (license plates)
names: ['license_plate']
"""

with open('car_plate_data/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

print("dataset.yaml updated successfully!")


dataset.yaml updated successfully!


In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Train the model using the car plate detection dataset
model.train(data='car_plate_data/dataset.yaml', epochs=10, imgsz=640)

# Save the trained model weights
model.save('yolov8_car_plate_model.pt')


100%|██████████| 6.25M/6.25M [00:00<00:00, 75.8MB/s]


Ultralytics 8.3.11 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=car_plate_data/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

100%|██████████| 755k/755k [00:00<00:00, 14.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/car_plate_data/labels/train... 433 images, 0 backgrounds, 0 corrupt: 100%|██████████| 433/433 [00:00<00:00, 542.19it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/10         0G      1.554      1.961      1.312          1        640: 100%|██████████| 28/28 [05:55<00:00, 12.70s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [01:39<00:00,  7.10s/it]

                   all        433        471      0.813      0.713      0.801      0.424



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/10         0G      1.474      1.695      1.281          1        640: 100%|██████████| 28/28 [05:54<00:00, 12.65s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  64%|██████▍   | 9/14 [00:57<00:34,  6.94s/it]

In [ ]:
import pandas as pd

# Define the student data as a list of dictionaries
student_data = [
    {'PlateNumber': 'ABC123', 'StudentName': 'John Doe'},
    {'PlateNumber': 'XYZ789', 'StudentName': 'Jane Smith'},
    {'PlateNumber': 'LMN456', 'StudentName': 'Alice Johnson'}
]

# Convert the list of dictionaries to a DataFrame
database = pd.DataFrame(student_data)

# Display the DataFrame to verify
print(database.head())


  PlateNumber    StudentName
0      ABC123       John Doe
1      XYZ789     Jane Smith
2      LMN456  Alice Johnson


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 22.9 MB/s eta 0:00:00


In [ ]:

import easyocr
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from ultralytics import YOLO

# Initialize the EasyOCR reader (supports multiple languages, but we'll use English)
reader = easyocr.Reader(['en'])

# Manually input the student data
student_data = {
    'PlateNumber': ['ABC123', 'XYZ789', 'LMN456', 'RHS8501'],  # Example plate numbers
    'StudentName': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Robert Green']  # Corresponding student names
}

# Convert the dictionary to a DataFrame
database = pd.DataFrame(student_data)

# Display the student database to verify
print(database.head())

# Load your trained YOLOv8 model
model = YOLO('yolov8_car_plate_model.pt')

# Upload an image for testing
uploaded_image = files.upload()
image_path = list(uploaded_image.keys())[0]
print(f"Testing on: {image_path}")

# Run inference with YOLOv8
results = model(image_path)
result = results[0]  # Access the first result

# Load the image using OpenCV
image = cv2.imread(image_path)

def preprocess_plate(image, box):
    """Preprocess the detected plate for better OCR."""
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped_plate = image[y1:y2, x1:x2]

    # Convert to grayscale
    gray_plate = cv2.cvtColor(cropped_plate, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding for better contrast
    processed_plate = cv2.adaptiveThreshold(
        gray_plate, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )

    # Resize the plate to improve OCR accuracy
    processed_plate = cv2.resize(processed_plate, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    return processed_plate

def extract_license_text(image, box):
    """Extract text from the preprocessed license plate using EasyOCR."""
    processed_plate = preprocess_plate(image, box)

    # Display the preprocessed plate (optional)
    plt.imshow(processed_plate, cmap='gray')
    plt.axis('off')
    plt.show()

    # Use EasyOCR to extract text
    result = reader.readtext(processed_plate)

    if len(result) > 0:
        # Extract the first detected text (highest confidence)
        plate_text = result[0][1]
    else:
        plate_text = ""

    return plate_text.strip()

# Main logic to extract text and match with the student database
if len(result.boxes) > 0:
    for box in result.boxes:
        # Extract the text from the license plate
        plate_text = extract_license_text(image, box)
        print(f"Extracted Plate Text: {plate_text}")

        # Search for the extracted plate number in the manually inputted data
        student_record = database[database['PlateNumber'].str.strip() == plate_text]

        if not student_record.empty:
            student_name = student_record.iloc[0]['StudentName']
            print(f"Student Name: {student_name}")
        else:
            print("Not a number plate.")
else:
    print("No license plate detected.")
